## Notebook Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import KFold
import tensorflow as tf
import os
import math
from datetime import timedelta
tf.random.set_seed(
1234
)

## Configure Variables

In [ ]:
# Windowing configuration specifies the number of previous time steps (WINDOW_SIZE) 
# and future time steps (HORIZON) to consider for each data sample.
WINDOW_SIZE = 5
HORIZON = 1

# Define the target variable for prediction and a list of feature column names that the model will use as inputs. 
# These columns represent various metrics that could potentially influence the target variable.
TARGET_COLUMN = 'panss_total'
FEATURE_COLUMNS = ['Mood', 'Sleep', 'Psychosis', 'Social', 'Anxiety',
                   'Hometime', 'Entropy', 'Screen_Duration',
                   'PHQ9', 'GAD', 'sfs_composite', 'PSQI']
# Combine the feature columns and the target column into a single list that will be used for data processing.
ALL_COLUMNS = FEATURE_COLUMNS + [TARGET_COLUMN]

# Establish the fractions of the dataset to be allocated to the training, validation, and test sets. 
train_frac = 0.70
validation_frac = 0.15
test_frac = 0.15

# Additionally, assert that the sum of the proportions equals 1, or 100% of the data, which ensures that no data is unaccounted for or duplicated across sets.
assert train_frac + validation_frac + test_frac == 1

# Set a variability threshold and minimum column count to filter out sequences without significant change.
# Due to high missingness in the dataset, forward fill imputation could skew the learning process due to a lack of meaningful variation.
VARIABILITY_THRESHOLD = 2
VARIABILITY_COLUMN_COUNT = 2


## Load File into Dataframe

In [ ]:
impute_df = pd.read_csv('forwardfill.csv')
impute_df.columns

In [ ]:
# Selecting columns relevant to our analysis and filtering the dataset.
# Among the columns, we are including participant identifiers, site information, and date values in addition to:
# Active Features:
# - EMA Surveys: 'Mood', 'Social', 'Sleep', 'Psychosis', 'Anxiety'
# - Structured Surveys: 'PHQ9', 'GAD', 'PSQI'
# Passive Features:
# - Smartphone Generated: 'Hometime', 'Entropy', 'Screen_Duration'
# Target Scores + Subscores
# - PANSS: 'panss_total', 'panss_neg', 'panss_pos', 'panss_gen'
# - SFS: 'sfs_scaled', 'social_engagement', 'interpersonal_behavior', 'prosocial_activities', 'recreation', 'competence', 'performance', 'employment'
df = impute_df[['Date', 'IID', 'site',
                 'Mood', 'Social', 'Sleep', 'Psychosis', 'Anxiety',
                 'Hometime', 'Entropy', 'Screen_Duration', 
                 'PHQ9', 'GAD', 'PSQI', 
                 'sfs_scaled', 'social_engagement', 'interpersonal_behavior', 'prosocial_activities', 'recreation', 'competence', 'performance', 'employment',
                 'panss_total', 'panss_neg', 'panss_pos', 'panss_gen']]

# After selecting our columns, we drop any rows that contain null values which is vital for the accuracy of the LSTM model.
df = df.dropna()
# Additionally, we standardize the 'Date' column to a consistent format to avoid parsing issues later on in the analysis.
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

len(df['IID'].unique())

## Data Preprocessing

In [210]:
# Initialize the MinMaxScaler to normalize feature values between 0 and 1
X_scaler = MinMaxScaler()
Y_scaler = MinMaxScaler()

# Define a function to partition time-series data for a participant into training, validation, and testing sets
def split_time_series_data(participant_data, train_frac, validation_frac):
    # Ensure the data is sorted chronologically to maintain the sequence integrity.
    participant_data_sorted = participant_data.sort_values('Date')

    # Determine the split indices based on the specified fractions.
    total_records = len(participant_data_sorted)
    train_end = int(train_frac * total_records)
    validation_end = train_end + int(validation_frac * total_records)

    # Divide the dataset into the respective sets.
    train_data = participant_data_sorted.iloc[:train_end]
    validation_data = participant_data_sorted.iloc[train_end:validation_end]
    test_data = participant_data_sorted.iloc[validation_end:]

    return train_data, validation_data, test_data

# Define a function to create input-output sequence pairs for LSTM training, based on variability in the data
def prepare_sequences(data):
    X, y = [], []
    for i in range(len(data) - WINDOW_SIZE - HORIZON):
        window = data.iloc[i:(i + WINDOW_SIZE + HORIZON)][ALL_COLUMNS]
        target_values = window[TARGET_COLUMN]

        # Analyze the standard deviation to identify columns with significant variability.
        columns_meeting_threshold = np.sum(window.std() > VARIABILITY_THRESHOLD)

        # Include a sequence if the target variable changes or if sufficient variability in column values is observed.
        if (target_values.nunique() > 1) or (columns_meeting_threshold >= VARIABILITY_COLUMN_COUNT):
            X.append(window.iloc[:-HORIZON - 1][FEATURE_COLUMNS].values)
            y.append(window.iloc[-1][TARGET_COLUMN])
    return X, y

# Create empty lists to hold the data splits for all participants.
X_train, X_validation, X_test = [], [], []
y_train, y_validation, y_test = [], [], []

# Obtain a list of unique participant identifiers.
participants = df['IID'].unique() 

# Process the data for each participant individually.
for participant in participants:
    # Select the data for the current participant.
    participant_data = df[df['IID'] == participant]

    # Apply the data splitting function to obtain training, validation, and testing sets.
    train_data, validation_data, test_data = split_time_series_data(participant_data, train_frac, validation_frac)

    # Generate sequences for each partition of the data.
    X_tr, y_tr = prepare_sequences(train_data)
    X_val, y_val = prepare_sequences(validation_data)
    X_te, y_te = prepare_sequences(test_data)

    # Append the generated sequences to the corresponding aggregate lists.
    X_train.extend(X_tr)
    y_train.extend(y_tr)
    X_validation.extend(X_val)
    y_validation.extend(y_val)
    X_test.extend(X_te)
    y_test.extend(y_te)


# Convert the sequence lists into numpy arrays for compatibility with machine learning models.
X_train = np.array(X_train)
y_train = np.array(y_train)
X_validation = np.array(X_validation)
y_validation = np.array(y_validation)
X_test = np.array(X_test)
y_test = np.array(y_test)    

# Instantiate a MinMaxScaler to normalize the features.
scaler = MinMaxScaler(feature_range=(0, 1))
# Fit the scaler on the training data, ensuring all data is stacked to match the scaler's expected input format.
scaler.fit(np.vstack(X_train))

# Apply the fitted scaler to normalize the training, validation, and test data.
X_train_scaled = scaler.transform(np.vstack(X_train)).reshape(X_train.shape)
X_validation_scaled = scaler.transform(np.vstack(X_validation)).reshape(X_validation.shape)
X_test_scaled = scaler.transform(np.vstack(X_test)).reshape(X_test.shape)

# Fit and transform the target variable using the scaler, and then flatten the resulting array.
y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
y_validation_scaled = scaler.transform(y_validation.reshape(-1, 1)).flatten()
y_test_scaled = scaler.transform(y_test.reshape(-1, 1)).flatten() 

# Output basic metrics regarding the size of each data split for review.
print(f" X_train: " + str(len(X_train)))
print(f" X_validation: " + str(len(X_validation)))
print(f" X_test: " + str(len(X_test)))

print(f" y_train: " + str(len(y_train)))
print(f" y_validation: " + str(len(y_validation)))
print(f" y_test: " + str(len(y_test)))

 X_train: 5157
 X_validation: 649
 X_test: 433
 y_train: 5157
 y_validation: 649
 y_test: 433


## Model Definitions

In [ ]:
# Setting batch size and buffer size for data shuffling and batching
batch_size = 110
buffer_size = 2048

# Preparing the training data
train_data = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_scaled))
# Caching the dataset for performance, shuffling it for randomness, and batching for training
train_data = train_data.cache().shuffle(buffer_size).batch(batch_size).repeat()

# Preparing the validation data in a similar way to the training data
val_data = tf.data.Dataset.from_tensor_slices((X_validation_scaled, y_validation_scaled))
val_data = val_data.cache().shuffle(buffer_size).batch(batch_size).repeat()

In [ ]:
# Defining a simple RNN model for baseline comparison
rnn_model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(4, input_shape=X_train.shape[-2:]), # Input layer with SimpleRNN
    tf.keras.layers.Dense(20, activation='relu'), # Hidden layer with ReLU activation
    tf.keras.layers.Dense(1) # Output layer with linear activation for regression
])
# Compiling the RNN model with mean squared error loss function and Adam optimizer
rnn_model.compile(loss='mean_squared_error', optimizer='adam')
# Displaying the model architecture for review
rnn_model.summary() 

In [ ]:
# Defining a more complex LSTM model to potentially capture more complex patterns
lstm_model = tf.keras.models.Sequential([
    # Bidirectional LSTM layer to process input in both directions
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, return_sequences=True),
                                 input_shape=X_train.shape[-2:]),
    tf.keras.layers.Dense(20, activation='relu'), # Dense layer with ReLU activation
    # Second Bidirectional LSTM layer to further process the sequence data
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(20, activation='relu'), # Additional dense layers with ReLU activation
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.25), # Dropout for regularization to prevent overfitting
    tf.keras.layers.Dense(units=1), # Output layer with linear activation for regression
])
# Compiling the LSTM model with mean squared error loss function and Adam optimizer
lstm_model.compile(optimizer='adam', loss='mse')
# Displaying the LSTM model architecture for review
lstm_model.summary()

## Model Training

In [ ]:
# Define the path where the best model will be saved
model_path = 'PANSS_Total_LSTM_wind5_horz1.h5'  

# Set up an EarlyStopping callback to prevent overfitting. Training will stop if 'val_loss' doesn't improve for 15 epochs.
early_stopings = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=15, 
    verbose=1, 
    mode='min'
)

# Set up a ModelCheckpoint callback to save the model with the lowest 'val_loss' after each epoch.
checkpoint =  tf.keras.callbacks.ModelCheckpoint(
    model_path, 
    monitor='val_loss', 
    save_best_only=True, 
    mode='min', 
    verbose=0
)

# Combine the callbacks into a list to be used in the model fitting process.
callbacks = [early_stopings, checkpoint]

# Fit the LSTM model to the training data, using the validation data for evaluation and the defined callbacks for early stopping and checkpointing.
history = lstm_model.fit(
    train_data, 
    epochs=150, 
    steps_per_epoch=100, 
    validation_data=val_data, 
    validation_steps=100, 
    callbacks=callbacks
)

In [ ]:
# Plotting the training and validation loss over epochs
plt.figure(figsize=(16,9))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'validation loss'])
plt.show() 

## Assessing Model Performance

In [ ]:
# Define the number of folds for cross-validation
num_folds = 10
# Initialize a KFold object with shuffle to ensure randomness and a fixed random state for reproducibility
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize lists to store results of metrics for each fold
rmse_values = []
mae_values = []
mape_values = []

# The following cross-validation is performed on the test set and considered as the final evaluation 
# step since the model parameters were not tuned based on this data. This evaluation method is separate 
# from the typical cross-validation approach which often partitions the training set.

# Iterate through each fold, splitting the data into training and testing sets for final evaluation
for train_index, test_index in kf.split(X_test_scaled):
    # Use the indices to partition the scaled test set
    X_train_fold, X_test_fold = X_test_scaled[train_index], X_test_scaled[test_index]
    y_train_fold, y_test_fold = y_test_scaled[train_index], y_test_scaled[test_index]

    # Load your pre-trained model (the path should point to the model file)
    model = tf.keras.models.load_model('PANSS_Total_LSTM_wind5_horz1.h5')

    # Use the model to make predictions on the test fold
    pred_fold = model.predict(X_test_fold)

    # Inverse transform the predictions to original scale using the scaler fitted on the training data
    pred_fold_inverse = scaler.inverse_transform(pred_fold)

    # Reshape y_test_fold to 2D array to match the shape expected by the scaler
    y_test_fold_2d = y_test_fold.reshape(-1, 1)

    # Inverse transform the actual test fold values to original scale
    actual_fold_inverse = scaler.inverse_transform(y_test_fold_2d)

    # Calculate Root Mean Squared Error (RMSE) for the current fold
    rmse_fold = np.sqrt(np.mean(np.square(np.subtract(actual_fold_inverse, pred_fold_inverse))))
    # Calculate Mean Absolute Error (MAE) for the current fold
    mae_fold = np.mean(np.abs(np.subtract(actual_fold_inverse, pred_fold_inverse)))
    # Calculate Mean Absolute Percentage Error (MAPE) for the current fold
    #    np.maximum utilized in the denominator to avoid division by 0 errors
    mape_fold = np.mean(np.abs(np.divide(np.subtract(actual_fold_inverse, pred_fold_inverse), 
                                         np.maximum(np.abs(actual_fold_inverse), 1)))) * 100

    # Append the calculated metrics to their respective lists
    rmse_values.append(rmse_fold)
    mae_values.append(mae_fold)
    mape_values.append(mape_fold)

# Calculate the mean and standard deviation of metrics across all folds
average_rmse = np.mean(rmse_values)
average_mae = np.mean(mae_values)
average_mape = np.mean(mape_values)
std_rmse = np.std(rmse_values)
std_mae = np.std(mae_values)
std_mape = np.std(mape_values)

# Print the results to summarize the cross-validation performance
print(f"Average RMSE across {num_folds} folds: {average_rmse}")
print(f"Average MAE across {num_folds} folds: {average_mae}")
print(f"Average MAPE across {num_folds} folds: {average_mape}")
print(f"Standard Deviation of RMSE across {num_folds} folds: {std_rmse}")
print(f"Standard Deviation of MAE across {num_folds} folds: {std_mae}")
print(f"Standard Deviation of MAPE across {num_folds} folds: {std_mape}")